**Problem:**

Company receives order to supply 1000 units

|                    | Machine_1 | Machine_2 | Machine_3 |
|--------------------|-----------|-----------|-----------|
| Variable Cost/unit |   2.39    |   1.99    |    2.99   | 
| Fixed Cost         |   300     |   250     |    400    | 
| Capacity           |   400     |   550     |    600    | 


Xi = number of units produced on machine i

Yi = 1 if machine i is used, 0 otherwise

In [1]:
import gurobipy as gp
from gurobipy import Model, GRB

# Create a new model
model = gp.Model("MultiProductFixedCost")

# Set product from machines
machines = ['Machine_1', 'Machine_2', 'Machine_3']

# Set parameters
variable_cost = {'Machine_1': 2.39, 'Machine_2': 1.99, 'Machine_3': 2.99}
fixed_cost = {'Machine_1': 300, 'Machine_2': 250, 'Machine_3': 400}
capacity = {'Machine_1': 400, 'Machine_2': 550, 'Machine_3': 600}
order = 1000 # to supply 1000 units

# Set decision variables: INTEGER, CONTINUOUS, BINARY
X = model.addVars(machines,lb=0, vtype=GRB.CONTINUOUS, name='Prod')
Y = model.addVars(machines,lb=0, vtype=GRB.BINARY, name='Setup')

# Set objective function GRB.MAXIMIZE
model.setObjective(
    gp.quicksum(variable_cost[m]*X[m] + fixed_cost[m]*Y[m] for m in machines),
    GRB.MINIMIZE
)

# Add constraints
model.addConstr(gp.quicksum(X[m] for m in machines) == order, 'Order')
for m in machines:
    model.addConstr(X[m] <= capacity[m]*Y[m], f'{m}_Capacity')

# Optimize the model
model.optimize()

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 19.6.0 19H2026)

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4 rows, 6 columns and 9 nonzeros
Model fingerprint: 0xcda464c6
Variable types: 3 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [2e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Presolve removed 4 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 3090 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.090000000000e+03, best bound 3.090000000000e+03, gap 0.0000%


In [2]:
print(f"ObjVal = {model.ObjVal:.2f}")
model.printAttr('X')

ObjVal = 3090.00

    Variable            X 
-------------------------
Prod[Machine_2]          550 
Prod[Machine_3]          450 
Setup[Machine_2]            1 
Setup[Machine_3]            1 
